In [ ]:
import pandas as pd
from keras import Sequential
from keras.src.callbacks import ModelCheckpoint
from keras.src.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go

In [ ]:
data = pd.read_csv("audi.csv").drop("model", axis=1)

test_labels = data["price"][-1000:-1]
labels = data["price"][0:-1000]

data = data.drop("price", axis=1)
data = pd.get_dummies(data, columns=["transmission"], dtype=int)
data = pd.get_dummies(data, columns=["fuelType"], dtype=int)
data

In [ ]:
scaler = StandardScaler()
data = scaler.fit_transform(data)
test_data = data[-1000:-1]
data = data[:-1000]

In [ ]:
model = Sequential()
model.add(Dense(units=32, input_shape=(11,)))
model.add(Dense(units=8, activation="relu"))
model.add(Dense(units=1, activation="relu"))
model.summary()

In [ ]:
mc = ModelCheckpoint("best_model_audi.keras", monitor="val_mae", save_best_only=True, mode="min", verbose=1)

model.compile(optimizer="adam",
              loss="mse",
              metrics=["mae"])

history = model.fit(data, labels, validation_split=0.25, callbacks=[mc], epochs=20, batch_size=32)

model.save("model_after_training_audi.keras")

In [ ]:
def draw_history(history):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["mae"], name="mae"))
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["val_mae"], name="val_mae"))
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["loss"], name="loss"))
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["val_loss"], name="val_loss"))
    fig.show()


draw_history(history)

In [ ]:
model.load_weights("model_after_training_audi.keras")

preds = model.predict(test_data, verbose=0)

df = pd.DataFrame(
    {"price": test_labels, "predictions": preds.ravel(), "error": abs(test_labels - preds.ravel())}
).reset_index().drop("index", axis=1)

print(f"Mean absolute error: {mean_absolute_error(df['price'], df['predictions'])}")

df